In [1]:
from process_chronicles_lbTag import *


### lb tag format

In [2]:
Purm_Louw_04 = '1791_Purm_Louw_04'
Mege_Loef = '1795_Mege_Loef'
Brus_Anon = '1802_Brus_Anon'

pl04 = ChronicleBucket(excel_path=f'../data/raw/{Purm_Louw_04}.xlsx', 
                       xml_path=f'../data/raw/{Purm_Louw_04}.xml')

ml = ChronicleBucket(excel_path=f'../data/raw/{Mege_Loef}.xlsx', 
                       xml_path=f'../data/raw/{Mege_Loef}.xml')

ba = ChronicleBucket(excel_path=f'../data/raw/{Brus_Anon}.xlsx', 
                       xml_path=f'../data/raw/{Brus_Anon}.xml')


/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
meta_date = pl04.meta_datum
# meta_all = pl04.meta_all
meta_locatie = pl04.meta_locatie
xml = pl04.meta_xml

In [7]:
# all abs in this xml file
# abs = xml.find_all('ab')

In [6]:
# the following def is also in helper.py

def join_line(line):
    # line: [<datum>1775</datum>, ' tot', <datum> 1788 </datum>, 'enz:;']
    joined = ''
    for i in line:
        try:
            if i.name in ['datum','locatie']:
                joined += f"<b>{i.text.strip(' ')}</b>"
            else:
                joined += f"{i.text.strip(' ')}"
        except:
            joined += f"{i.strip(' ')}"
    return joined

# join_line(d_ab[20]['line'])

def decompose_ab(ab):
    # input: single ab
    # output: 
    ab_dict = {}
    now_ind = None
    now_facs = ''

    for ind,i in enumerate(ab):
        if i.name == 'lb':
            now_ind = ind
            now_facs = i['facs']
            ab_dict[(now_ind,now_facs)] = [] # {'facs':i['facs'],'line':[]}
        else:
            try:
                ab_dict[(now_ind,now_facs)].append(i)  # [now_ind]['line'].append(i)
            except:
                pass
    return ab_dict

# d_ab = decompose_ab(abs[0])


In [7]:
# d_abs = []
# for ab in abs:
#     _ = {ab['facs']:decompose_ab(ab)}
#     d_abs.append(_)


In [10]:
# len(d_abs) 265 abs 
# join_line(d_abs[0][(9, '#facs_3_r1l5')])
# given from excel: page 17, line r3l5
#facs_17_r3l5
page = 17
line = 'r3l5'
region = 'r3'

lb_id = f"#facs_{page}_{line}"
ab_id = f"#facs_{page}_{region}"
ab_ind = None

for i in range(len(d_abs)):
    assert len(d_abs[i].keys()) ==1, print(d_abs[i].keys())
    if list(d_abs[i].keys())[0] == ab_id:
        ab_ = d_abs[i]
        ab_ind = i  # assign ab_ind, the position of this ab among all abs in this file

for ii,i in enumerate(sorted(ab_[ab_id])):
    if i[1] == lb_id:
        r = list(range(ii-6,ii+5))
print(r,ab_ind)

[-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8] 27


In [16]:
output_line = []
for i in r:
    # print(i)
    if i < 0:
        try:
            assert len(d_abs[ab_ind-1].values())==1
            _ab = list(d_abs[ab_ind-1].keys())[0]
            lbss = d_abs[ab_ind-1][_ab] # dict
            lbs = sorted([k for k in lbss.keys()]) # key list
            output_line += lbss[lbs[i]]
            # print(list(d_abs[ab_ind-1].values())[0][_k[i]])
        except:
            print(ab_ind)


    elif i > len(d_abs[ab_ind][ab_id])-1:
        try:
            assert len(d_abs[ab_ind+1].values())==1
            _k = sorted([k for k in list(d_abs[ab_ind+1].values())[0].keys()])
            output_line += list(d_abs[ab_ind+1].values())[0][_k[i]]
            # print(list(d_abs[ab_ind+1].values())[0][_k[i]])
        except:
            print(ab_ind)
    else:
        _k = sorted([k for k in list(d_abs[ab_ind].values())[0].keys()])
        output_line += list(d_abs[ab_ind].values())[0][_k[i]]
        # print(list(d_abs[ab_ind].values())[0][_k[i]])


In [27]:
# find following def in helper.py
def find_index(d_abs, page, line, region, n):
    # n: how much context do you want, forwards/backwards n lines
    lb_id = f"#facs_{page}_{line}"
    ab_id = f"#facs_{page}_{region}"
    ab_ind = None

    for i in range(len(d_abs)):
        assert len(d_abs[i].keys()) ==1, print(d_abs[i].keys())
        if list(d_abs[i].keys())[0] == ab_id:
            ab_ = d_abs[i]
            ab_ind = i  # assign ab_ind, the position of this ab among all abs in this file

    for ii,i in enumerate(sorted(ab_[ab_id])):
        if i[1] == lb_id:
            r = list(range(ii-(n+1),ii+n))
    # print(r,ab_ind)
    return r, ab_ind, ab_id

def generate_output_line(d_abs, r, ab_ind, ab_id):
    # r: a range list, in the middle among them is the text we look for
    # and we are going to find the context of that text (forward X lines, backwards X lines)
    # ab_ind: the position of this ab among all abs in this file
    # in case if we are going to find a previous ab for context
    output_line = []
    for i in r:
        if i < 0:
            try:
                # if ab_ind = 0 , you cannot go backwards in this ab
                # if ab_ind - 1 <=0, then start from 0 # pass
                if ab_ind - 1 <= 0:
                    pass
                else:
                    assert len(d_abs[ab_ind-1].values())==1
                    _ab = list(d_abs[ab_ind-1].keys())[0]
                    lbss = d_abs[ab_ind-1][_ab] # dict
                    lbs = sorted([k for k in lbss.keys()]) # key list
                    output_line += lbss[lbs[i]]
                    # print(list(d_abs[ab_ind-1].values())[0][_k[i]])
            except:
                print(ab_ind)


        elif i > len(d_abs[ab_ind][ab_id])-1:
            try:
                assert len(d_abs[ab_ind+1].values())==1
                _k = sorted([k for k in list(d_abs[ab_ind+1].values())[0].keys()])
                output_line += list(d_abs[ab_ind+1].values())[0][_k[i]]
                # print(list(d_abs[ab_ind+1].values())[0][_k[i]])
            except:
                print(ab_ind)
        else:
            _k = sorted([k for k in list(d_abs[ab_ind].values())[0].keys()])
            output_line += list(d_abs[ab_ind].values())[0][_k[i]]
            # print(list(d_abs[ab_ind].values())[0][_k[i]])
    return output_line


In [4]:
d_abs = pl04.get_all_abs()
lb_orders = pl04.get_all_lbs_order()

In [ ]:
# {'belong_to_date': {'date_line': '#facs_20_r2l3', 'text': 'tus 1786'},
#  'lb': '#facs_21_r1l10',
#  'line': 'r1l10',
#  'page': 21,
#  'type': 'locatie',
#  'value': 'Purmerend',
#  'whole_line_context': 'Dan zal Ons Purmerend,'}

In [21]:
# process xml data of each chronicle
# generate a dictionary 
# {'#facs_3_r1l10': {'type': 'datum'},
#  '#facs_5_r4l4': {'type': 'locatie',
#   'belong_to_date': {'text': ' 1788 ', 'date_line': '#facs_3_r1l10'}}, ... }
ab_locatie_date_dic = {}
locatie_count_ab = 0
abs = xml.find_all('ab')
for ab in abs:
    for ind,i in enumerate(ab):
        if i.name in ['locatie','datum']:
            lb_fac = i.findPrevious('lb')['facs'] 
            ab_locatie_date_dic[lb_fac] = {'type':i.name}
        if i.name == 'locatie':
            locatie_count_ab += 1
            try:
                ab_locatie_date_dic[lb_fac].update({'belong_to_date':{'text':i.find_previous('datum').text, 
                                                             'date_line':i.find_previous('datum').findPrevious('lb')['facs']}})
            except:
                print(i)

 

<locatie>P U R M E R E N D E</locatie>


In [4]:
test = pl04.locatie_date_dict(n=5)

<locatie>P U R M E R E N D E</locatie>


In [5]:
test['#facs_21_r3l3']

{'type': 'locatie',
 'raw_text': 'Purmerende',
 'detail': {'page': '21', 'region': 'r3', 'line': 'r3l3'},
 'belong_to_date': {'raw_text': 'tus 1786',
  'date_lb_name': '#facs_20_r2l2',
  'normed_date': '1786-08-xx',
  'date_locatie_d': 53},
 'hover': "Jae! Burgers, Wij zijn Vreij,\nEn kennen geen bedwang,\nDat elk zijn Zeeges\nAan dit vrije Volks woordhang.\nUijt Vriendsc#2\nKronijk van<b>Purmerende</b>\n<b>1778</b>\n<b>27 Junij</b>Alhier is in de Stad aangekomen de\nNuntius van den Paus te<b>Romen</b>residee¬\nrende om de belangens van 't Roomsche Gees¬\ntelijke Hof waar te nemen in de<b>Oostenrijk¬</b>\n"}

In [6]:
locatie_count = 0
threshold = 50
date_locatie_d_smaller_count = 0
for lb,values in test.items():
    if values['type'] == 'locatie':
        locatie_count += 1
    try:
        if values['belong_to_date']['date_locatie_d'] <= threshold:
            date_locatie_d_smaller_count += 1
    except:
        pass

print(locatie_count)
print(date_locatie_d_smaller_count)

640
610


In [10]:
# locatie_lbs = [k for k,v in locatie_data_dict.items() if v['type'] == 'locatie']
# datum_lbs = [k for k,v in locatie_data_dict.items() if v['type'] == 'datum']


In [22]:
def get_locatie():
    # from chronicle's excel file, get all locatie lb_names, corresponding page, region, lines 
    # return dict()
    # {'#facs_3_r1l5': {'page': 3, 'line': 'r1l5', 'region': 'r1'},...}
    locatie_lb_names = {}
    for i in meta_locatie.index:
        page = meta_locatie.loc[i,'Page']
        line = meta_locatie.loc[i,'Line']
        region = meta_locatie.loc[i,'Region']
        _ = f"#facs_{page}_{line}"
        locatie_lb_names[_] = {'page':page,'line':line,'region':region}

    return locatie_lb_names

In [165]:
locatie_data_dict, _ =  pl04.find_locatie_date()

<locatie>P U R M E R E N D E</locatie>


In [21]:
d_abs = pl04.get_all_abs()
locaties = [k for k,v in locatie_data_dict.items() if v['type']=='locatie']
n =3

for ab in d_abs:
    for ab_name, lbs in ab.items():
        for lb_name, lb in lbs.items():
            for i in locaties:
                if lb_name[1] == i:
                    try:
                        page, line = i.split('_',2)[1:3]
                        page,region = locatie_data_dict[i]['detail']['ab_name'].split('_',2)[1:3]
                        r, ab_ind,ab_id = find_index(d_abs, page, line, region, n)
                        o = generate_output_line(d_abs, r, ab_ind, ab_id)
                        hover_info = join_line(o)
                        locatie_data_dict[i]['detail'] = {'page':page,'region':region,'line':line}
                        locatie_data_dict[i]['hover'] = hover_info
                    except:
                        print(i)


# {'#facs_3_r1l10': {'type': 'datum'},
#  '#facs_5_r4l4': {'type': 'locatie',
#   'belong_to_date': {'text': ' 1788 ', 'date_line': '#facs_3_r1l10'}}, ... }

In [7]:
mapping_location = pl04.mapping_loacation

In [11]:
test

{'#facs_3_r1l5': {'type': 'locatie',
  'raw_text': 'P U R M E R E N D E',
  'detail': {'page': '3', 'region': 'r1', 'line': 'r1l5'},
  'hover': 'V e r v o l g\nd e r\nK R O N Y K \nv a n\n<b>P U R M E R E N D E</b>.\nDerde Deel\nEerste Stuk.\nBevattende de Jaaren\nv a n \n<b>1775</b>tot<b>1788</b>enz:;'},
 '#facs_3_r1l10': {'type': 'datum',
  'raw_text': ' 1788 ',
  'detail': {'ab_name': '#facs_3_r1'}},
 '#facs_5_r4l4': {'type': 'locatie',
  'raw_text': 'Purmerende',
  'detail': {'page': '5', 'region': 'r4', 'line': 'r4l4'},
  'belong_to_date': {'raw_text': ' 1788 ',
   'date_lb_name': '#facs_3_r1l10',
   'normed_date': '1788-xx-xx'},
  'hover': "den, de goede Wille en de oplettend¬\nheijd van een Geschiedschrijver, altoos#\nin het ooge Werd gehouden.Voorreeden.\nAan de Waarde, en Weldenkende Leezer!\nZie hier weder, Een vervolg der Kronijk der\nStadt<b>Purmerende</b>, Heeft Uwe in de voorige dee¬\nlen der Beschreijvinge van deeze Stad, iets\ngevonden, Het geene Aanmerkings waar¬\ndig 

In [12]:
all_lo = 0
get_lo = 0
for k,v in test.items():
    if v['type'] == 'locatie':
        all_lo += 1
        if mapping_location.get(v['raw_text'].strip(' ').lower()):
            get_lo += 1
print(all_lo,get_lo)

640 518


In [28]:
mapping_location.get(locatie_data_dict['#facs_5_r4l4']['raw_text'].strip(' ').lower())

{'match_place': 'Purmerende', 'la': 52.509903, 'lo': 4.946108}

In [24]:
all_locatie_count = 0
have_norm = 0
no_norm = []
for k,v in locatie_data_dict.items():
    if v['type'] == 'locatie':
        all_locatie_count += 1
        try:
            _ = v['belong_to_date']['date_lb_name'].split('_',2)[1:3]
            date_page = int(_[0])
            date_line = _[1]    

            if len(meta_date.query("Page ==@ date_page").query("Line==@ date_line")['datum'].tolist()) >0:
            #     have_norm += 1
                have_norm += 1
            else:
                no_norm.append(v['belong_to_date']['date_lb_name'])

        except:
            # print(k)
            pass
            


In [30]:
print(all_locatie_count,have_norm)
print(len(no_norm),len(set(no_norm)))

640 321
318 141


In [31]:
set(no_norm)

{'#facs_100_line_1568225964842_818',
 '#facs_101_line_1568226273645_986',
 '#facs_104_line_1568227340237_1495',
 '#facs_105_line_1568227632437_1626',
 '#facs_106_line_1568227846686_1720',
 '#facs_106_line_1568227949030_1780',
 '#facs_107_line_1568228110546_1850',
 '#facs_107_line_1568228241269_1913',
 '#facs_108_line_1568228559471_2047',
 '#facs_108_r2l6',
 '#facs_109_r1l5',
 '#facs_109_r2l4',
 '#facs_110_line_1568229260544_63',
 '#facs_110_r1l6',
 '#facs_111_r2l3',
 '#facs_112_r2l6',
 '#facs_113_line_1568230779609_486',
 '#facs_113_line_1568230928810_553',
 '#facs_114_line_1568231089099_638',
 '#facs_114_line_1568231238937_710',
 '#facs_115_r5l2',
 '#facs_117_line_1568232511751_1226',
 '#facs_117_line_1568232669334_1310',
 '#facs_119_r2l6',
 '#facs_120_line_1568233571499_1662',
 '#facs_120_line_1568233809093_1766',
 '#facs_121_line_1568234008671_1860',
 '#facs_121_r3l3',
 '#facs_122_line_1568234331040_2016',
 '#facs_122_r2l2',
 '#facs_123_line_1568235070732_2156',
 '#facs_124_line_156

In [36]:
# date_page, _,date_line = 
_ = locatie_data_dict['#facs_5_r4l4']['belong_to_date']['date_line'].split('_',2)[1:3]
date_page = int(_[0])
date_line = _[1]
date_value = locatie_data_dict['#facs_5_r4l4']['belong_to_date']['text']
# date_norm = meta_date.query("Page ==@ date_page").query("Line==@ date_line").query("Value==@date_value")
# date_norm

,Value,Context,Imagename,Doc,Page,Region,Line,Word,datum,offset,length,continued,year,month,day
1,1788,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1788-xx-xx,8,6,False,1788,01,01


In [34]:
c = 0
for i in meta_date['datum'].tolist():
    if 'x' in str(i):
        c += 1
c

68

In [10]:
page = 132
region = 'r1'
line = 'r1l1'
# <lb facs="#facs_132_r1l1" n="N002"/>Kronijk van <locatie>Purmerende</locatie>
r, ab_ind,ab_id = find_index(d_abs, page, line, region, 5)
o = generate_output_line(d_abs, r, ab_ind, ab_id)
print(join_line(o))

Zal Wagten tot in<b>Maij</b>Wanneer de Ver¬
anderinge der Vroedschappen door de Com¬
missarissen uijt Naam van Z D H: alhier
gedaan zal zijn blijvende de tegenwoordige
Heeren dat Ampt tot dien tijd bekleeden,219
Kronijk van<b>Purmerende</b>
<b>1788</b>
<b>14 April</b>
de Missive van Z D H, wegens de
veranderinge van de Regeering of
Wel der Vroedschappen dezer Stad, is



In [11]:
meta_date.head()

,Value,Context,Imagename,Doc,Page,Region,Line,Word,datum,offset,length,continued,year,month,day
0,1775,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1775-xx-xx,0,4,False,1775,01,01
1,1788,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1788-xx-xx,8,6,False,1788,01,01
2,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0012.jpg,901446,12,region_1568800990264_4443,line_1568800996505_4451,NaN,1785-xx-xx,21,4,False,1785,01,01
3,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0013.jpg,901446,13,r2,r2l3,NaN,1785-xx-xx,21,4,False,1785,01,01
4,1785,merende in 't Jaar 1785.,NL-HlmNHA_143_21_0014.jpg,901446,14,r1,r1l14,NaN,1785-xx-xx,19,4,False,1785,01,01


In [94]:
# t = xml.find_all('title')[0].get_text()
df_date_clean = meta_date.loc[meta_date['datum'].str.contains('x')==False,:]
print(meta_date.shape, df_date_clean.shape)   # (487, 7) (418, 7)

(487, 15) (417, 15)


In [95]:
# px.histogram(df_date_clean.loc[:,['datum','year']],x='datum')
# px.histogram(df_date_clean.loc[:,['datum','month']],x='datum',xbins=604800000)
import plotly.graph_objects as go
fig_histo = go.Figure()
fig_histo.add_trace(go.Histogram(x=df_date_clean['datum'],
                                 xbins = dict(size=2419200000)))
fig_histo.update_layout(
    title_text=f'date time frequency', # title of plot
    xaxis_title_text='date (each bin is a month)', # xaxis label
    yaxis_title_text='Count'
)

fig_histo.show()

In [67]:
# find date tag first
# accroding to the excel metadata, sort and filter continous date tag
# find the text between each date tag
# extract the lacatie/text in these text

In [96]:
meta_date.head()

,Value,Context,Imagename,Doc,Page,Region,Line,Word,datum,offset,length,continued,year,month,day
0,1775,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1775-xx-xx,0,4,False,1775,01,01
1,1788,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1788-xx-xx,8,6,False,1788,01,01
2,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0012.jpg,901446,12,region_1568800990264_4443,line_1568800996505_4451,NaN,1785-xx-xx,21,4,False,1785,01,01
3,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0013.jpg,901446,13,r2,r2l3,NaN,1785-xx-xx,21,4,False,1785,01,01
4,1785,merende in 't Jaar 1785.,NL-HlmNHA_143_21_0014.jpg,901446,14,r1,r1l14,NaN,1785-xx-xx,19,4,False,1785,01,01


In [85]:
'#facs_22_line_1568066442046_348'.split('_',2)

['#facs', '22', 'line_1568066442046_348']

In [159]:
# test['#facs_132_r1l1']
count_all_locatie = 0
count_locatie_with_date = 0
count_locatie_with_normalized_date = 0
failed_date_lb = []
for k,v in test.items():
    if v['type'] == 'locatie':
        count_all_locatie += 1
        if 'belong_to_date' in v.keys():
            count_locatie_with_date += 1
            date_lb = v['belong_to_date']['date_lb_name']
            _,page,line = date_lb.split('_',2)
            # print(meta_date.loc[(meta_date['Page']==int(page))&(meta_date['Line']==line),'datum'].tolist())
            if meta_date.loc[(meta_date['Page']==int(page))&(meta_date['Line']==line),'datum'].tolist():
                count_locatie_with_normalized_date += 1
            else:
                failed_date_lb.append(v['belong_to_date']['date_lb_name'])

print(count_all_locatie)
print(count_locatie_with_date)
print(count_locatie_with_normalized_date)


640
639
321


In [162]:
set(failed_date_lb)

{'#facs_100_line_1568225964842_818',
 '#facs_101_line_1568226273645_986',
 '#facs_104_line_1568227340237_1495',
 '#facs_105_line_1568227632437_1626',
 '#facs_106_line_1568227846686_1720',
 '#facs_106_line_1568227949030_1780',
 '#facs_107_line_1568228110546_1850',
 '#facs_107_line_1568228241269_1913',
 '#facs_108_line_1568228559471_2047',
 '#facs_108_r2l6',
 '#facs_109_r1l5',
 '#facs_109_r2l4',
 '#facs_110_line_1568229260544_63',
 '#facs_110_r1l6',
 '#facs_111_r2l3',
 '#facs_112_r2l6',
 '#facs_113_line_1568230779609_486',
 '#facs_113_line_1568230928810_553',
 '#facs_114_line_1568231089099_638',
 '#facs_114_line_1568231238937_710',
 '#facs_115_r5l2',
 '#facs_117_line_1568232511751_1226',
 '#facs_117_line_1568232669334_1310',
 '#facs_119_r2l6',
 '#facs_120_line_1568233571499_1662',
 '#facs_120_line_1568233809093_1766',
 '#facs_121_line_1568234008671_1860',
 '#facs_121_r3l3',
 '#facs_122_line_1568234331040_2016',
 '#facs_122_r2l2',
 '#facs_123_line_1568235070732_2156',
 '#facs_124_line_156

In [182]:
for i in d_abs[1].values():
    print(i.keys())

dict_keys([(1, '#facs_4_r1l1'), (3, '#facs_4_r1l2'), (6, '#facs_4_r1l3'), (9, '#facs_4_r1l4'), (12, '#facs_4_r1l5'), (15, '#facs_4_r1l6'), (18, '#facs_4_r1l7'), (21, '#facs_4_r1l8'), (24, '#facs_4_r1l9'), (27, '#facs_4_r1l11'), (30, '#facs_4_r1l13'), (33, '#facs_4_r1l14'), (36, '#facs_4_r1l15'), (39, '#facs_4_r1l17'), (42, '#facs_4_r1l18'), (45, '#facs_4_r1l20'), (48, '#facs_4_r1l21'), (51, '#facs_4_r1l22'), (54, '#facs_4_r1l24'), (57, '#facs_4_r1l26'), (60, '#facs_4_r1l27')])


In [185]:
# '#facs_100_line_1568225964842_818'
test['#facs_22_line_1568066442046_348']

{'type': 'datum', 'raw_text': '2 Dec', 'detail': {'ab_name': '#facs_22_r1'}}

In [201]:
ab_locatie_date_dic = {}
locatie_count_ab = 0
date_no_norm = []
abs = xml.find_all('ab')
for ab in abs:
    for ind,i in enumerate(ab):
        if i.name in ['locatie','datum']:
            lb_fac = i.findPrevious('lb')['facs'] 
            ab_locatie_date_dic[lb_fac] = {'type':i.name,'raw_text':i.text,
                                            'detail':{'ab_name':ab['facs']}}
        if i.name == 'locatie':
            locatie_count_ab += 1
            # but if try failed, the actual count of locatie with a belonged date would be less than this count here
            try:
                raw_text = i.find_previous('datum').text
                date_lb_name = i.find_previous('datum').findPrevious('lb')['facs']

                # find normalized date from excel metadata
                _ = date_lb_name.split('_',2)[1:3]
                date_page = int(_[0])
                date_line = _[1]
                if meta_date.loc[(meta_date['Page']==int(date_page))&(meta_date['Line']==date_line),'datum'].tolist():
                    normed_date = meta_date.loc[(meta_date['Page']==int(date_page))&(meta_date['Line']==date_line),'datum'].tolist()[-1]
                else:
                    # failed_date_lb.append(v['belong_to_date']['date_lb_name'])
                    # print(f'nodate: {date_lb_name}')
                    new_lb = i.find_previous('datum').find_previous('datum').findPrevious('lb')['facs']
                    _ = new_lb.split('_',2)[1:3]
                    date_page = int(_[0])
                    date_line = _[1]
                    try:
                        normed_date = meta_date.loc[(meta_date['Page']==int(date_page))&(meta_date['Line']==date_line),'datum'].tolist()[-1]
                    except:
                        normed_date = None

                ab_locatie_date_dic[lb_fac].update({'belong_to_date':{'raw_text':raw_text, 
                                                                      'date_lb_name':date_lb_name,
                                                                      'normed_date':normed_date}})
                
            except:
                # print(i)
                pass

In [157]:
d = list([datetime.datetime.strptime(i,'%Y-%m-%d') for i in df_date_clean['datum'].tolist()])
d = [datetime.datetime.strftime(i,'%Y-%m-%d') for i in d]
c = [str(i) for i in range(len(df_date_clean['datum'].tolist()))]
_ = pd.DataFrame()
_['d'] = d
_['c'] = c
f = px.scatter(_,x=c,y=d)
f.update_layout(
    # title="Plot Title",
    xaxis_title="chronicle context order (the N th appeared date tag)",
    yaxis_title="date")
f


## new format : l tag

In [1]:
from process_chronicles_lTag import *

In [2]:
Purm_Louw_04 = '1796_Purm_Louw_04'
Mege_Loef = '1795_Mege_Loef'
Brus_Anon = '1802_Brus_Anon'

pl04 = ChronicleBucket(xml_path=f'../../l_tag_format_chronicles/Kronieken XML nieuw/{Purm_Louw_04}.xml')

ml = ChronicleBucket(xml_path=f'../../l_tag_format_chronicles/Kronieken XML nieuw/{Mege_Loef}.xml')

ba = ChronicleBucket(xml_path=f'../../l_tag_format_chronicles/Kronieken XML nieuw/{Brus_Anon}.xml')


In [3]:
import re
xml = pl04.meta_xml # #facs_3_r1l10
lorder = pl04.l_order() 
mapping_location = pl04.mapping_location
# for i in xml.find_all(name='datum',datum=re.compile("\d{4}-\d{2}-\w{2}")):
#     print(i)

In [4]:
locatie_date_dict, summary_dict = pl04.locatie_date_pack(nlines=5)

In [5]:
locatie_date_dict['#facs_18_r2l7']

{'locatie_origin': 'Nederland',
 'locatie_match': {'match_place': 'Nederland',
  'la': 52.2858356,
  'lo': 5.6549386},
 'locatie_date_l': '#facs_17_r3l5',
 'normal_date': '1786-07-12',
 'locatie_date_distance': 62,
 'locatie_lines': "'S Lands tegenspoed bepaalen:\nWant word der Burg'ren Arm\nHet Waepentuijg ontrukt\nEn zal d' Aristocraat\nIn 't Eijnd een zegenpraelen,\nDan Word gantsch <b>Nederland</b>,\nWeer in Slavernij gedrukt;\nZoudt Gij, dan naar de Stem\nVan 't Vaederland niet hooren,\nDen plicht niet agten.\nDie aan ijder Mensch betaamt"}

In [12]:
print(summary_dict['all_locatie_count'])
print(summary_dict['locatie_with_date_count'])
print(summary_dict['locatie_matched'])
# print(summary_dict[''])
# len([i for i in locate_date_distance if i <10])

799
774
635


In [13]:
timeline = []
for k,v in locatie_date_dict.items():
    timeline.append(v['normal_date'])

_ = pd.DataFrame()
_['d'] = timeline
_['c'] = [str(i) for i in range(len(timeline))]
f = go.Scatter(x = [str(i) for i in range(len(timeline))],
               y = timeline,
               mode='markers',
               name = 'ccc')
# f = px.scatter(_,x='c',y='d')
fig = go.Figure(data=f)
fig.update_layout(
    # title="Plot Title",
    xaxis_title="chronicle context order (the N th appeared date tag)",
    yaxis_title="date")
fig

In [103]:
timeline = []
for l in xml.find_all('l'):
    try:
        d = l.find(name='datum',datum = re.compile("\d{4}-\d{2}-\w{2}"))
        if d:
            timeline.append(d['datum'])
    except:
        pass

_ = pd.DataFrame()
_['d'] = timeline
_['c'] = [str(i) for i in range(len(timeline))]
f = go.Scatter(x = [str(i) for i in range(len(timeline))],
               y = timeline,
               mode='markers',
               name = 'ccc')
# f = px.scatter(_,x='c',y='d')
fig = go.Figure(data=f)
fig.update_layout(
    # title="Plot Title",
    xaxis_title="chronicle context order (the N th appeared date tag)",
    yaxis_title="date")
fig

In [9]:
# all_date_tag = []
all_range_date = []
xml = pl04.meta_xml
for l in xml.find_all('l'):
    for ele in l:
        try:   #### 1770 to 1815
            if ele.name == 'datum':
                norm_date = ele['datum']
                if re.match("\d{4}-\d{2}-\w{2}", norm_date):
                    # all_date_tag.append(norm_date)
                    y,m,d = norm_date.split('-')
                    if 1770 <= int(y) <= 1815:
                        all_range_date.append(norm_date)  
                        ele['in_range'] = 'yes'         
        except:
            pass


In [11]:
for l in xml.find_all('l'):
    for ele in l:
        try:   #### 1770 to 1815
            if ele.name == 'datum':
                print(ele)
        except:
            pass

<datum datum="1775-xx-xx">1775</datum>
<datum datum="1788-xx-xx"> 1788 </datum>
<datum datum="1785-xx-xx">1785</datum>
<datum datum="1785-xx-xx">1785</datum>
<datum datum="1785-xx-xx">1785</datum>
<datum datum="1786-07-12" in_range="yes">12 Julij A<hi rend="superscript:true;">o</hi> 1786</datum>
<datum datum="1786-07-12" in_range="yes">12 Julij 1786</datum>
<datum datum="1786-08-xx" in_range="yes">Augus¬</datum>
<datum datum="1786-08-xx" in_range="yes">tus 1786</datum>
<datum datum="1778-06-27" in_range="yes">1778</datum>
<datum datum="1778-06-27" in_range="yes">27 Junij </datum>
<datum datum="1778-07-03" in_range="yes">3 Julij </datum>
<datum datum="1778-07-xx" in_range="yes">Maand Julij</datum>
<datum datum="1789-03-30" in_range="yes">30 Maart 1789</datum>
<datum datum="1778-06-16" in_range="yes">1778</datum>
<datum datum="1778-06-16" in_range="yes">16 Junij</datum>
<datum datum="1778-12-02" in_range="yes">2 Dec</datum>
<datum datum="1779-01-10" in_range="yes">1779</datum>
<datum dat

In [11]:
# find locatie's previous date (in a monthly resolution)

locate_date_distance = []
all_locatie_count = 0
locatie_with_date_count = 0
locatie_matched = 0
locatie_date_dict = {}



xml = pl04.meta_xml
for l in xml.find_all('l'):
    for ele in l:
        if ele.name == 'locatie':
            all_locatie_count += 1
            try:
                locatie_date = ele.findPrevious(name='datum',datum = re.compile("\d{4}-\d{2}-\w{2}"))
                if locatie_date:
                    locatie_with_date_count += 1
                    normaled_date = locatie_date['datum']

                    locatie_date_l = locatie_date.findPrevious(name='l')['facs']
                    d = lorder[l['facs']]-lorder[locatie_date_l]
                    ma = mapping_location.get(ele.text.strip().lower())
                    if ma:
                        locatie_matched += 1
                    locatie_date_dict[l['facs']] = {"locatie_origin": ele.text.strip(),
                                                    "locatie_match": ma,
                                                    "locatie_date_l": locatie_date_l,
                                                    "normal_date": normaled_date,
                                                    "locatie_date_distance":d,
                                                    "locatie_lines":findNlines(l,n=3)}

                    # print(f"location l: {l['facs']}")
                    # print(f"date l: {locatie_date_l}")
                    # print(f"date: {normaled_date}")
                    # print('_______')
                    locate_date_distance.append(lorder[l['facs']]-lorder[locatie_date_l])
            except:
                pass



In [15]:
locatie_date_dict['#facs_18_r2l7']
# print(f"all locatie: {all_locatie_count}")
# print(f"locatie with date: {locatie_with_date_count}")      
# print(f"locatie matched: {locatie_matched}")     

{'locatie_origin': 'Nederland',
 'locatie_match': {'match_place': 'Nederland',
  'la': 52.2858356,
  'lo': 5.6549386},
 'locatie_date_l': '#facs_17_r3l5',
 'normal_date': '1786-07-12',
 'locatie_date_distance': 62,
 'locatie_lines': "Het Waepentuijg ontrukt\nEn zal d' Aristocraat\nIn 't Eijnd een zegenpraelen,\nDan Word gantsch <b>Nederland</b>,\nWeer in Slavernij gedrukt;\nZoudt Gij, dan naar de Stem\nVan 't Vaederland niet hooren,"}

In [11]:
fig = go.Figure(data=[go.Histogram(x=summary_dict['locatie_date_distance'])])
fig.update_layout(
    title_text='location-date distance', 
    xaxis_title_text='distance (how many lines between the location and its date)', 
    yaxis_title_text='frequency',
)
fig.show()

# all the chronicles

In [6]:
import os,glob
from process_chronicles_lTag import *
from collections import Counter


In [7]:
o_dict = {}
fig_data = []
for f in glob.glob('../../l_tag_format_chronicles/Kronieken XML nieuw/*xml'):
    chronicle_id = os.path.basename(f).split('.')[0]
    chronicle = ChronicleBucket(xml_path=f)
    xml = chronicle.meta_xml
    f = generate_timeline(xml_name=chronicle_id, xml=xml)
    # fig_data.append(f)
    # _ = chronicle.filter_date_range()
    locatie_date_dict, summary_dict = chronicle.locatie_date_pack(nlines=5)
    o_dict[chronicle_id] = {"summary":summary_dict,"locatie_date":locatie_date_dict}
    print(f"{chronicle_id} finished")
    

1795_Mege_Loef finished
1797_Brst_Mori finished
1796_Antw_Aars finished
1802_Venl_Post finished
1796_Leeu_Buma finished
1795_Maas_Anon finished
1793_Bred_Ouko finished
1802_Brus_Anon finished
1787_Gent_Anon_01 finished
1790_Mech_Wijn finished
1796_Purm_Louw_04 finished
1808_Gent_Quic finished
1797_Alkm_Anon finished
1796_Purm_Louw_05 finished
1795_Zalt_Kist finished
1792_Brie_Klui_10 finished
1792_Brie_Klui_11 finished


In [8]:
norm_date_dict = {k:[] for k,v in o_dict.items()}

for chronicle_id, v in o_dict.items():
    for f,locatie_date in v['locatie_date'].items():
        norm_date_dict[chronicle_id].append(locatie_date['normal_date'])
        

In [9]:
for k,v in norm_date_dict.items():
    print(k,len(v),len(set(v)))

1795_Mege_Loef 231 157
1797_Brst_Mori 124 10
1796_Antw_Aars 242 85
1802_Venl_Post 1235 677
1796_Leeu_Buma 171 83
1795_Maas_Anon 299 34
1793_Bred_Ouko 197 18
1802_Brus_Anon 696 203
1787_Gent_Anon_01 141 34
1790_Mech_Wijn 1200 230
1796_Purm_Louw_04 692 171
1808_Gent_Quic 909 420
1797_Alkm_Anon 357 175
1796_Purm_Louw_05 873 127
1795_Zalt_Kist 232 124
1792_Brie_Klui_10 296 86
1792_Brie_Klui_11 147 53


In [10]:
o_dict['1796_Antw_Aars']['locatie_date']['#facs_3_r1l12']

{'locatie_origin': 'Steenhoudersvest',
 'locatie_match': None,
 'locatie_date_l': '#facs_3_r1l8',
 'normal_date': '1796-03-16',
 'locatie_date_distance': 4,
 'locatie_lines': 'sedert\n1794 15 Maert tot 16 Maert 1796\nbeschreven door\nJan Van Aarsen\nStoofmaker en Koperslager\nop de <b>Steenhoudersvest</b>\n<b>te Antwerpen</b>.\n@@@@achte@\nH a a l  b o e k & \n(Dit wild zeggen:\nHael-boek-(van stoven.)'}

In [16]:
o_dict['1795_Mege_Loef']['summary'].keys()

dict_keys(['locatie_date_distance', 'all_locatie_count', 'locatie_with_date_count', 'locatie_matched'])

In [26]:
cc= 0
for f,v in o_dict['1796_Antw_Aars']['locatie_date'].items():
    if v['locatie_match']:
        # print(v['locatie_lines'])
        cc += 1

In [21]:
for fa,v in o_dict['1796_Antw_Aars']['locatie_date'].items():
    try:
        print(v['locatie_match']['la'])
    except:
        pass
    # if v['locatie_match']:
    #     try:
    #         print('thisone:',v['locatie_match']['locatie_origin'])
    #     except:
    #         pass \n

51.221248
52.169581
51.221248
51.6017723
52.169581
51.221248
51.028057
53.199063
53.199063
52.169581
50.928327
50.928327
53.199063
52.169581
51.459117
53.603069
nan
52.169581
51.221248
52.890631
51.459117
51.221248
53.199063
51.221248
51.459117
51.459117
53.199063
53.199063
51.401598
51.221248
53.199063
48.8566969
51.221248
53.199063
53.199063
51.865943
54.063861
51.221248
51.05355
53.199063
53.199063
53.199063
51.221248
53.199063
53.199063
50.846675
51.05355
53.199063
51.221248
53.199063
53.199063
51.276861958879465
51.294538701413245
51.276861958879465
51.294538701413245
51.221248
53.199063
53.199063
53.199063
53.199063
53.199063
53.199063
51.276861958879465
51.638413
53.199063
51.393323894135996
53.199063
53.199063
53.199063
53.199063
50.838107527866654
51.225891
51.847301
53.199063
50.838107527866654
53.199063
50.008614
53.199063
51.580689723584676
51.797565
53.199063
53.199063
50.646094
53.199063
51.221248
53.199063
50.846675
50.879307
50.879307
51.517228
46.603354
53.199063
51.92

In [17]:
o_dict['1796_Antw_Aars']['locatie_date']

{'#facs_3_r1l12': {'locatie_origin': 'Steenhoudersvest',
  'locatie_match': None,
  'locatie_date_l': '#facs_3_r1l8',
  'normal_date': '1796-03-16',
  'locatie_date_distance': 4,
  'locatie_lines': 'sedert\n1794 15 Maert tot 16 Maert 1796\nbeschreven door\nJan Van Aarsen\nStoofmaker en Koperslager\nop de <b>Steenhoudersvest</b>\n<b>te Antwerpen</b>.\n@@@@achte@\nH a a l  b o e k & \n(Dit wild zeggen:\nHael-boek-(van stoven.)'},
 '#facs_3_r1l13': {'locatie_origin': 'te Antwerpen',
  'locatie_match': None,
  'locatie_date_l': '#facs_3_r1l8',
  'normal_date': '1796-03-16',
  'locatie_date_distance': 5,
  'locatie_lines': '1794 15 Maert tot 16 Maert 1796\nbeschreven door\nJan Van Aarsen\nStoofmaker en Koperslager\nop de <b>Steenhoudersvest</b>\n<b>te Antwerpen</b>.\n@@@@achte@\nH a a l  b o e k & \n(Dit wild zeggen:\nHael-boek-(van stoven.)\n@ @ @'},
 '#facs_4_r1l10': {'locatie_origin': 'Mers Straat',
  'locatie_match': None,
  'locatie_date_l': '#facs_3_r1l8',
  'normal_date': '1796-03-16

In [26]:
f_list= []


for k,v in norm_date_dict.items():
    _ = pd.DataFrame()
    _['d'] = v
    _['c'] = [str(i) for i in range(len(v))]
    f = go.Scatter(x = [str(i) for i in range(len(v))],
                y = v,
                mode='markers',
                name = k)
    f_list.append(f)
fig = go.Figure(data=f_list)
fig.update_layout(
    # title="Plot Title",
    xaxis_title="chronicle context order (the N th appeared date tag)",
    yaxis_title="date")
fig

In [11]:
figs = go.Figure(data=fig_data)
figs.update_layout(
    # title="Plot Title",
    xaxis_title="chronicle context order (the N th appeared date tag)",
    yaxis_title="date")
figs

In [12]:
figs.write_html('../plots/date_range.html')

## Geomatch

In [3]:
import glob,re,os
import pandas as pd
from collections import Counter

In [15]:
target_chronicles_names = []
for i in glob.glob('../../l_tag_format_chronicles/Kronieken_XML_nieuw/*.xml'):
    chronicle_id = os.path.basename(i).split('.')[0]
    target_chronicles_names.append(chronicle_id)

In [16]:
len(target_chronicles_names)

17

In [17]:
target_chronicles_names

['1795_Mege_Loef',
 '1797_Brst_Mori',
 '1796_Antw_Aars',
 '1802_Venl_Post',
 '1796_Leeu_Buma',
 '1795_Maas_Anon',
 '1793_Bred_Ouko',
 '1802_Brus_Anon',
 '1787_Gent_Anon_01',
 '1790_Mech_Wijn',
 '1796_Purm_Louw_04',
 '1808_Gent_Quic',
 '1797_Alkm_Anon',
 '1796_Purm_Louw_05',
 '1795_Zalt_Kist',
 '1792_Brie_Klui_10',
 '1792_Brie_Klui_11']

In [18]:
geo_match = pd.read_excel('../data/geomatching_annotations_V1.xlsx',engine='openpyxl', sheet_name='2792482 Leuven')


/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [19]:
geo_match['Annotatie'] = geo_match['Annotatie'].astype(str)

In [20]:
# columns 
# place context author match Annotatie 
geo_names = [i.strip('\\') for i in geo_match['author'].unique()]
[i for i in target_chronicles_names if i in geo_names]


['1795_Mege_Loef',
 '1797_Brst_Mori',
 '1796_Antw_Aars',
 '1802_Venl_Post',
 '1796_Leeu_Buma',
 '1795_Maas_Anon',
 '1802_Brus_Anon',
 '1808_Gent_Quic',
 '1797_Alkm_Anon',
 '1795_Zalt_Kist']

In [21]:
o = [i for i in target_chronicles_names if i in geo_names]
o = ['\\'+i for i in o]
Counter([i for i in geo_match['author'].tolist() if i in o])

Counter({'\\1802_Brus_Anon': 5,
         '\\1808_Gent_Quic': 4,
         '\\1795_Maas_Anon': 23,
         '\\1795_Mege_Loef': 1,
         '\\1802_Venl_Post': 13,
         '\\1797_Alkm_Anon': 5,
         '\\1795_Zalt_Kist': 2,
         '\\1797_Brst_Mori': 3,
         '\\1796_Leeu_Buma': 1,
         '\\1796_Antw_Aars': 1})

In [23]:
geo_match.head()

,place,context,author,match,Annotatie,Opm
0,rijp,Enckhuisen al waeren thuis gecomen;\n Item den...,\1652_Rijp_Dirc,.,nan,NaN
1,.,.,.,rip (Pakistan; 68.7915; 24.7824),nan,NaN
2,.,.,.,rip (Pakistan; 69.35414; 25.11708),nan,NaN
3,.,.,.,rip (Pakistan; 68.98019; 25.87435),nan,NaN
4,.,.,.,rip (Senegal; -15.5; 13.66667),nan,NaN


In [24]:
place_range_ind = []
for i in geo_match.index:
    if geo_match.iloc[i,0]!= '.':    # place
        place_range_ind.append(i)

break_geo = []
for i in range(len(place_range_ind)):
    try:
        s = place_range_ind[i]
        e = place_range_ind[i+1]
        break_geo.append(geo_match.iloc[s:e,:])
    except:
        assert i == 999
        s = place_range_ind[i]
        e = len(geo_match)
        break_geo.append(geo_match.iloc[s:e,:])

In [31]:
break_geo[2]

,place,context,author,match,Annotatie,Opm
46,Vlaenderen,79\n Ghereetschaep oock ghemaeck hebbende; om ...,\1668_Gent_Bill_08,.,nan,NaN
47,.,.,.,vlaanderen (Belgium; Arr. Mechelen; 4.5; 51.0),x,NaN
48,.,.,.,genderen (Netherlands; Midden-Noord-Brabant; E...,nan,NaN
49,.,.,.,panderen (Netherlands; Arnhem/Nijmegen; Herwen...,nan,NaN
50,.,.,.,zenderen (Netherlands; Twente; Borne; 6.72361;...,nan,NaN
51,.,.,.,lendewarden (Latvia; Pierīga; 24.837521; 56.71...,nan,NaN
52,.,.,.,landen (Belgium; Arr. Leuven; 5.080476; 50.75567),nan,NaN
53,.,.,.,ederen (Germany; Düren; 6.24744; 50.94588),nan,NaN
54,.,.,.,ladbergen (Germany; Steinfurt; 7.736438; 52.13...,nan,NaN
55,.,.,.,gandersen (Germany; Northeim; 10.025543; 51.87...,nan,NaN


In [30]:
len(set(break_geo[0]['Annotatie'].tolist()))

2

In [56]:
geo_match_dict = {}
for b in break_geo:
    place = b['place'].tolist()[0]
    geo = b.loc[b['Annotatie'] != 'nan','match'].tolist()
    if geo:
        if 'LOKAAL' in geo:
            pass
        else:
            if 'ANDERS' in geo:
                geo = b.loc[:,'Annotatie'].tolist()
                geo = [i for i in geo if i != 'nan']

            if place in geo_match_dict.keys():
                geo_match_dict[place].append(geo)
            else:
                geo_match_dict[place] = []
                geo_match_dict[place].append(geo)

for k,v in geo_match_dict.items():
    geo_match_dict[k] = [i for i in v if i]

In [63]:
len([i for i in geo_match['Annotatie'].tolist() if i != 'nan'])

493

In [66]:
len(geo_match_dict)

321

In [35]:
# o_dict['Brie_Klui_11']['summary']['locatie_date_distance']
fig = go.Figure()
fig1 = go.Figure()
for k in o_dict.keys():
    fig.add_trace(go.Histogram(x = o_dict[k]['summary']['locatie_date_distance'],name=f"{k} {o_dict[k]['summary']['locatie_with_date_count']}"))
    fig1.add_trace(go.Histogram())
# fig = go.Figure(data=[go.Histogram(x=o_dict['Brie_Klui_11']['summary']['locatie_date_distance'])])

fig.update_layout(
    title_text='location-date distance', 
    xaxis_title_text='distance (how many lines between the location and its date)', 
    yaxis_title_text='frequency',
    barmode='stack'
)
fig.show()
fig.write_html('location_distance.html')

In [22]:
c = ['all_locatie_count', 'locatie_with_date_count', 'locatie_matched']
df_count = pd.DataFrame(columns=c)
for k in o_dict.keys():
    for i in c:
        df_count.loc[k,i] = o_dict[k]['summary'][i]
        # print(k, i, o_dict[k]['summary'][i])
    # print('_________')

In [23]:
fig = go.Figure()


,all_locatie_count,locatie_with_date_count,locatie_matched
Mege_Loef,412,411,128
Brst_Mori,151,138,58
Antw_Aars,271,271,154
Venl_Post,2113,2111,622
Leeu_Buma,199,199,110
Maas_Anon,2703,2632,1466
Bred_Ouko,213,213,113
Brus_Anon,767,763,283
Gent_Anon_01,208,189,76
Mech_Wijn,1314,1314,644
